<a href="https://colab.research.google.com/github/nikhil-mathews/MastersPr_Predicting-Human-Pathogen-PPIs-using-Natural-Language-Processing-methods/blob/main/Web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This has been implemented in mixed order and should only be used as a guide. Preliminary preprocessing has also been done at the end

In [ ]:
import pandas as pd

df = pd.read_csv("C:/Users/nik00/py/proj/humanPPI-ids-uniprot.txt", sep="\t", header=None)

In [ ]:
df[1].value_counts()

P05067    1356
P34932     582
P0CG48     546
Q9Y4K3     504
P08238     489
          ... 
O95645       1
Q8IXT1       1
Q13519       1
A2A368       1
Q5QPE5       1
Name: 1, Length: 17312, dtype: int64

### We start by getting all protein identifiers in test and train data

In [ ]:
x = set(pd.unique(df[0]))
x = x| set(pd.unique(df[1]))
len(x)

18502

In [ ]:
df = pd.read_csv("C:/Users/nik00/py/proj/hyppi-independent.txt", sep="\t")
df.head()
x = x| set(pd.unique(df.ID1))
len(x)
x = x| set(pd.unique(df.ID2))
len(x)

19731

In [ ]:

df = pd.read_csv("C:/Users/nik00/py/proj/hyppi-train.txt", sep="\t")
x = x| set(pd.unique(df.ID1))
len(x)
x = x| set(pd.unique(df.ID2))
len(x)

22648

In [ ]:
x = list(x)
x[1905]
len(set(x))

22648

In [ ]:
count = 0
for i in x:
    if len(i) >6:
        count+=1
        print(i)
        if count>20:
            break
count

A0A0H2W4J5
A0A151PNK0
A0A154T0N1
A0A151PQD1
A0A0H2W777
A0A151PNW1
A0A151PMG2
A0A0H2W4C7
A0A0H2W0E3
A0A154SWT5
A0A154T4S0
A0A151PLX2
A0A151PMN3
A0A154LY31
A0A154S8H9
A0A0H2W0N2
A0A0H2W8E0
A0A151PS41
A0A151PP88
A0A151PQ89
A0A1U8QWI5


21

In [ ]:
from Bio import SeqIO
l=[]
for seq_record in SeqIO.parse("C:/Users/nik00/py/proj/uniprot_sprot.fasta", "fasta"):
    l.append(seq_record.id.split('|')[1])

In [ ]:
count = 0
for i in ll:
    if i not in l:
        count+=1
count

1344

In [ ]:
from Bio import SeqIO
l=[]
for seq_record in SeqIO.parse("C:/Users/nik00/py/proj/uniprot_sprot.fasta", "fasta"):
#     print(seq_record.id)
#     print(repr(seq_record.seq))
#     print(len(seq_record))
    l.append(seq_record.id.split('|')[1])

In [ ]:
len(l)

563972

In [ ]:
'O00487' in l

True

In [ ]:
import requests as r
from Bio import SeqIO
from io import StringIO

cID='A0A151PMG2'

baseUrl="http://www.uniprot.org/uniprot/"
currentUrl=baseUrl+cID+".fasta"
response = r.post(currentUrl)
cData=''.join(response.text)

Seq=StringIO(cData)
pSeq=list(SeqIO.parse(Seq,'fasta'))

s = pSeq[0].seq
pSeq[0].description
s
#if no response,  len(pSeq) == 0

IndexError: list index out of range

In [ ]:
pSeq[0]
#gene pSeq[0].id.split('|')[2]
gene = pSeq[0].id.split('|')[2] #dont use gene
line = pSeq[0].description
line = line[line.find(gene)+len(gene)+1:]
Protein = line[:line.find('OS=')-1]
line = line[len(Protein)+1:]
OrganismName = line[3:line.find('OX=')-1]
OrganismIdentifier = line[3:line.find('OX=')-1]
line = line[line.find('OX='):]
OrganismIdentifier = line[3:line.find('GN=')-1]
line = line[line.find('GN='):]
GeneName = line[3:line.find('PE=')-1]
line = line[line.find('PE='):]
ProteinExistence = line[3:line.find('SV=')-1]
line = line[line.find('SV='):]
SequenceVersion = line[3:]
print('SequenceVersion',SequenceVersion)
print('ProteinExistence',ProteinExistence)
print('GeneName',GeneName)
print('OrganismName',OrganismName)
print('OrganismIdentifier',OrganismIdentifier)
print('Protein',Protein)

SequenceVersion 1
ProteinExistence 4
GeneName yiaD
OrganismName Yersinia pestis
OrganismIdentifier 632
Protein OmpA family lipoprotein


### Create DATA file that has several attributes for each protein we get by web scraping

In [ ]:
COLUMN_NAMES = ['id','Sequence','SequenceVersion','Protein','ProteinExistence','GeneName','OrganismName','OrganismIdentifier']
df = pd.DataFrame(columns=COLUMN_NAMES)
nf = []
df.to_pickle("C:/Users/nik00/py/proj/DATA.pkl")
df = pd.read_pickle("C:/Users/nik00/py/proj/DATA.pkl")
df.head()


,id,Sequence,SequenceVersion,Protein,ProteinExistence,GeneName,OrganismName,OrganismIdentifier


In [ ]:

import requests as r
from Bio import SeqIO
from io import StringIO

#extract data for each protein identifier
baseUrl="http://www.uniprot.org/uniprot/"
df = pd.read_pickle("C:/Users/nik00/py/proj/DATA.pkl")
count = -1

for i in x[df.shape[0]:]:
    count+=1
    currentUrl=baseUrl+i+".fasta"
    response = r.post(currentUrl)
    cData=''.join(response.text)
    Seq=StringIO(cData)
    pSeq=list(SeqIO.parse(Seq,'fasta'))
    if len(pSeq) == 0:
        nf.append(i)
        df = df.append({'id': i}, ignore_index=True)
        continue
    
    gene = pSeq[0].id.split('|')[2] #dont use gene
    line = pSeq[0].description
    line = line[line.find(gene)+len(gene)+1:]
    Protein = line[:line.find('OS=')-1]
    line = line[len(Protein)+1:]
    OrganismName = line[3:line.find('OX=')-1]
    OrganismIdentifier = line[3:line.find('OX=')-1]
    line = line[line.find('OX='):]
    OrganismIdentifier = line[3:line.find('GN=')-1]
    line = line[line.find('GN='):]
    GeneName = line[3:line.find('PE=')-1]
    line = line[line.find('PE='):]
    ProteinExistence = line[3:line.find('SV=')-1]
    line = line[line.find('SV='):]
    SequenceVersion = line[3:]    
    df = df.append({'id': i,'Sequence': pSeq[0].seq,'SequenceVersion':SequenceVersion, 'Protein':Protein, 'ProteinExistence':ProteinExistence, 'GeneName':GeneName, 'OrganismName':OrganismName, 'OrganismIdentifier':OrganismIdentifier}, ignore_index=True)
    
    if count%10 == 0:
        df.to_pickle("C:/Users/nik00/py/proj/DATA.pkl")
    
    
    
len(nf)

Q8NGG4


744

In [ ]:
#df.head(150)
'Q8NGG4'
df = pd.read_pickle("C:/Users/nik00/py/proj/DATA.pkl")
df.shape[0]
df.iloc[50:60,:]
df.head(150)

#df = df.set_index('id')
'A0A151PMG2' in list(df['id'])
df.shape[0]

22648

In [ ]:
outdated = []
#df = df.set_index('id')
outdated = df[df.isna().any(axis=1)].index
outdated

Index(['A0A151PNK0', 'A0A154T0N1', 'A0A151PQD1', 'A0A0H2W777', 'A0A151PNW1',
       'A0A151PMG2', 'A0A154SWT5', 'A0A154T4S0', 'A0A151PLX2', 'A0A151PMN3',
       ...
       'A0A151PRL8', 'A0A154M4U8', 'A0A154SM52', 'A0A154MGY1', 'A0A154SSZ8',
       'A0A154T663', 'A0A151PKU6', 'A0A154T0S4', 'A0A154LY99', 'A0A151PP82'],
      dtype='object', name='id', length=739)

In [ ]:
URL = 'https://www.uniprot.org/uniparc/?query='
i = 'A0A151PQD1'
page = r.get(URL+i)
content = str(page.content)
content = content[content.find('entryID')+7:]
link = content[content.find('<a href="')+9:content.find('">UPI')]
new_URL = 'https://www.uniprot.org'+link+'.fasta'
OS = content[content.find('<td>')+4:content.find('<br/>')]
print(OS)
new_URL

Yersinia pestis


'https://www.uniprot.org/uniparc/UPI0007A8A9E1.fasta'

In [ ]:
df.loc['A0A151PQD1']['OrganismName']

nan

In [ ]:
URL = 'https://www.uniprot.org/uniparc/?query='
#Get sequences of the outdated protein identifiers
for i in outdated:
    page = r.get(URL+i)
    content = str(page.content)
    content = content[content.find('entryID')+7:]
    link = content[content.find('<a href="')+9:content.find('">UPI')]
    new_URL = 'https://www.uniprot.org'+link+'.fasta'
    OS = content[content.find('<td>')+4:content.find('<br/>')]
    response = r.post(new_URL)
    cData=''.join(response.text)
    Seq=StringIO(cData)
    pSeq=list(SeqIO.parse(Seq,'fasta'))
    #print(pSeq[0].seq)
    if len(pSeq) == 0:
        nf.append(i)
        continue
    df.loc[i]['Sequence'] = pSeq[0].seq
    df.loc[i]['OrganismName'] = OS

In [ ]:
df = pd.read_pickle("C:/Users/nik00/py/proj/DATA.pkl")
'fsaffag' not in df.index

True

In [ ]:
pd.unique(df["OrganismName"])
#df.head()
count = 0
for i in df.index:
    count+=1
    if df.loc[i]["OrganismName"].find("Yersinia") != -1:
        df.loc[i]["OrganismName"] = 'Yersinia pestis'
    if df.loc[i]["OrganismName"].find("Homo sapiens") != -1:
        df.loc[i]["OrganismName"] = 'Homo sapiens'
    
count
pd.unique(df["OrganismName"])

array(['Homo sapiens', 'Yersinia pestis', 'Pan troglodytes (Chimpanzee)',
       'Salmonella enterica (Salmonella choleraesuis)',
       'Mus musculus (Mouse)', 'Salmonella houtenae',
       'Salmonella enterica I'], dtype=object)

In [ ]:
df.to_pickle("C:/Users/nik00/py/proj/DATA.pkl")

In [ ]:
import pandas as pd
data = pd.read_pickle("C:/Users/nik00/py/proj/DATA.pkl")
data.describe()

,Sequence,SequenceVersion,Protein,ProteinExistence,GeneName,OrganismName,OrganismIdentifier
count,22648,21909,21909,21909,21909,22648,21909
unique,21981,9,20094,6,19867,7,9
top,"(M, L, V, M, A, P, R, T, V, L, L, L, L, S, A, ...",1,Uncharacterized protein,1,,Homo sapiens,9606
freq,13,11660,360,15877,583,18622,18061


In [ ]:
len(list(data.loc["A6NFA1"]["Sequence"]))

517

#### Get amino acid sequences from train dataset and store it as a dataframe

In [ ]:
df = pd.read_csv("C:/Users/nik00/py/proj/hyppi-train.txt", sep="\t")
idd = df.loc[0]["ID1"]
df["ID1"]
organisms = []
for idd in df["ID2"]:
    if idd in data.index:
        organisms.append(data.loc[idd]["OrganismName"])
    else:
        print("Not found")
set(organisms)

{'Salmonella enterica (Salmonella choleraesuis)',
 'Salmonella enterica I',
 'Salmonella houtenae',
 'Yersinia pestis'}

In [ ]:
not_Yersinia=[]
for idd in df["ID2"]:
    org = data.loc[idd]["OrganismName"]
    if org != 'Yersinia pestis':
        not_Yersinia.append(idd)
not_Yersinia

['A0A151PRP0', 'A0A151PRL9', 'A0A151PRU0', 'A0A151PRN7', 'A0A151PRM1']

In [ ]:
l1=list("anad")
l2 = list("bnuqe")
' '.join(l1+l2)

'a n a d b n u q e'

In [ ]:
df.iloc[0]["ID1"]
data.loc[df.iloc[row]["ID1"]]["Sequence"]

Seq('MEDEEGPEYGKPDFVLLDQVTMEDFMRNLQLRFEKGRIYTYIGEVLVSVNPYQE...PSR')

In [ ]:
df["Human"] = [list(data.loc[df.iloc[row]["ID1"]]["Sequence"]) for row in range(df.shape[0])]
df["Yersinia"] = [list(data.loc[df.iloc[row]["ID2"]]["Sequence"]) for row in range(df.shape[0])]

In [ ]:
df["Joined"] = [df.loc[row]['Human']+df.loc[row]['Yersinia'] for row in range(df.shape[0])]

In [ ]:
df.head()

,label,ID1,ID2,Human,Yersinia,Joined
0,1,B0I1T2,Q7CJI0,"[M, E, D, E, E, G, P, E, Y, G, K, P, D, F, V, ...","[M, T, D, L, S, R, R, K, L, L, T, G, F, W, Q, ...","[M, E, D, E, E, G, P, E, Y, G, K, P, D, F, V, ..."
1,1,Q8IZ63,Q8D014,"[M, Q, H, P, K, P, F, C, A, P, A, A, P, Q, E, ...","[M, P, I, T, L, F, V, L, C, T, T, P, E, E, T, ...","[M, Q, H, P, K, P, F, C, A, P, A, A, P, Q, E, ..."
2,1,O75592,Q7CGI3,"[M, M, M, C, A, A, T, A, S, P, A, A, A, S, S, ...","[M, V, D, V, I, D, I, Q, A, Q, M, A, D, K, I, ...","[M, M, M, C, A, A, T, A, S, P, A, A, A, S, S, ..."
3,1,Q9HCI7,Q7CKV2,"[M, N, P, V, N, A, T, A, L, Y, I, S, A, S, R, ...","[M, T, M, T, T, F, S, H, P, S, L, I, P, L, D, ...","[M, N, P, V, N, A, T, A, L, Y, I, S, A, S, R, ..."
4,1,Q9BY50,Q8D0B6,"[M, V, R, A, G, A, V, G, A, H, L, P, A, S, G, ...","[M, A, S, T, T, M, G, V, K, L, D, E, A, T, R, ...","[M, V, R, A, G, A, V, G, A, H, L, P, A, S, G, ..."


In [ ]:
#print(data.loc['B0I1T2']["Sequence"])
' '.join(df.loc[0]['Human'])


MEDEEGPEYGKPDFVLLDQVTMEDFMRNLQLRFEKGRIYTYIGEVLVSVNPYQELPLYGPEAIARYQGRELYERPPHLYAVANAAYKAMKHRSRDTCIVISGESGAGKTEASKHIMQYIAAVTNPSQRAEVERVKDVLLKSTCVLEAFGNARTNRNHNSSRFGKYMDINFDFKGDPIGGHIHSYLLEKSRVLKQHVGERNFHAFYQLLRGSEDKQLHELHLERNPAVYNFTHQGAGLNMTVHSALDSDEQSHQAVTEAMRVIGFSPEEVESVHRILAAILHLGNIEFVETEEGGLQKEGLAVAEEALVDHVAELTATPRDLVLRSLLARTVASGGRELIEKGHTAAEASYARDACAKAVYQRLFEWVVNRINSVMEPRGRDPRRDGKDTVIGVLDIYGFEVFPVNSFEQFCINYCNEKLQQLFIQLILKQEQEEYEREGITWQSVEYFNNATIVDLVERPHRGILAVLDEACSSAGTITDRIFLQTLDMHHRHHLHYTSRQLCPTDKTMEFGRDFRIKHYAGDVTYSVEGFIDKNRDFLFQDFKRLLYNSTDPTLRAMWPDGQQDITEVTKRPLTAGTLFKNSMVALVENLASKEPFYVRCIKPNEDKVAGKLDENHCRHQVAYLGLLENVRVRRAGFASRQPYSRFLLRYKMTCEYTWPNHLLGSDKAAVSALLEQHGLQGDVAFGHSKLFIRSPRTLVTLEQSRARLIPIIVLLLQKAWRGTLARWRCRRLRAIYTIMRWFRRHKVRAHLAELQRRFQAARQPPLYGRDLVWPLPPAVLQPFQDTCHALFCRWRARQLVKNIPPSDMPQIKAKVAAMGALQGLRQDWGCRRAWARDYLSSATDNPTASSLFAQRLKTLQDKDGFGAVLFSSHVRKVNRFHKIRNRALLLTDQHLYKLDPDRQYRVMRAVPLEAVTGLSVTSGGDQLVVLHARGQDDLVVCLHRSRPPLDNRVGELVGVLAAHCQGEGRTLEVRVSDCIPLSHRGVRRLISVEPRPEQP

'M E D E E G P E Y G K P D F V L L D Q V T M E D F M R N L Q L R F E K G R I Y T Y I G E V L V S V N P Y Q E L P L Y G P E A I A R Y Q G R E L Y E R P P H L Y A V A N A A Y K A M K H R S R D T C I V I S G E S G A G K T E A S K H I M Q Y I A A V T N P S Q R A E V E R V K D V L L K S T C V L E A F G N A R T N R N H N S S R F G K Y M D I N F D F K G D P I G G H I H S Y L L E K S R V L K Q H V G E R N F H A F Y Q L L R G S E D K Q L H E L H L E R N P A V Y N F T H Q G A G L N M T V H S A L D S D E Q S H Q A V T E A M R V I G F S P E E V E S V H R I L A A I L H L G N I E F V E T E E G G L Q K E G L A V A E E A L V D H V A E L T A T P R D L V L R S L L A R T V A S G G R E L I E K G H T A A E A S Y A R D A C A K A V Y Q R L F E W V V N R I N S V M E P R G R D P R R D G K D T V I G V L D I Y G F E V F P V N S F E Q F C I N Y C N E K L Q Q L F I Q L I L K Q E Q E E Y E R E G I T W Q S V E Y F N N A T I V D L V E R P H R G I L A V L D E A C S S A G T I T D R I F L Q T L D M H H R H H L H Y T S R

In [ ]:
df.to_pickle("C:/Users/nik00/py/proj/hyppi-train.pkl")

In [ ]:
import numpy as np
try:
    with open("/content/drive/MyDrive/ML_Data/hyppi-train.pkl", "rb") as fh:
      df = pickle.load(fh)
except:
    df = pd.read_pickle("C:/Users/nik00/py/proj/hyppi-train.pkl")
df = df.sample(frac=1).reset_index(drop=True)[['Human','Yersinia','Joined','label']]   #SHUFFLE
df['label'] = np.where(df['label']==-1, 0, 1)
df.head()

,Human,Yersinia,Joined,label
0,"[M, K, D, K, Q, K, K, K, K, E, R, T, W, A, E, ...","[M, A, K, A, S, R, H, N, L, S, I, G, I, S, M, ...","[M, K, D, K, Q, K, K, K, K, E, R, T, W, A, E, ...",1
1,"[M, S, W, S, G, L, L, H, G, L, N, T, S, L, T, ...","[M, Q, H, V, T, G, S, K, R, R, L, I, G, W, V, ...","[M, S, W, S, G, L, L, H, G, L, N, T, S, L, T, ...",0
2,"[M, S, L, F, D, L, F, R, G, F, F, G, F, P, G, ...","[M, A, E, L, P, A, K, R, R, F, T, R, I, D, R, ...","[M, S, L, F, D, L, F, R, G, F, F, G, F, P, G, ...",1
3,"[M, A, V, A, P, R, L, F, G, G, L, C, F, R, F, ...","[M, R, I, F, A, I, S, C, S, S, Y, T, Y, C, Y, ...","[M, A, V, A, P, R, L, F, G, G, L, C, F, R, F, ...",0
4,"[M, S, T, I, Q, S, E, T, D, C, Y, D, I, I, E, ...","[M, S, Y, A, F, P, G, T, F, P, G, R, R, M, R, ...","[M, S, T, I, Q, S, E, T, D, C, Y, D, I, I, E, ...",0


In [ ]:
df.to_pickle("C:/Users/nik00/py/proj/hyppi-train.pkl")

#### Get amino acid sequences from test dataset and store it as a dataframe

In [ ]:
df = pd.read_csv("C:/Users/nik00/py/proj/hyppi-independent.txt", sep="\t")
idd = df.loc[0]["ID1"]
df["ID1"]
df
organisms = []
for idd in df["ID1"]:
    if idd in data.index:
        organisms.append(data.loc[idd]["OrganismName"])
    else:
        print("Not found")
set(organisms)

{'Homo sapiens'}

In [ ]:
df["Human"] = [list(data.loc[df.iloc[row]["ID1"]]["Sequence"]) for row in range(df.shape[0])]
df["Yersinia"] = [list(data.loc[df.iloc[row]["ID2"]]["Sequence"]) for row in range(df.shape[0])]
df["Joined"] = [df.loc[row]['Human']+df.loc[row]['Yersinia'] for row in range(df.shape[0])]

In [ ]:
df.to_pickle("C:/Users/nik00/py/proj/hyppi-independent.pkl")

In [ ]:
try:
    with open("/content/drive/MyDrive/ML_Data/hyppi-train.pkl", "rb") as fh:
      df = pickle.load(fh)
except:
    df = pd.read_pickle("C:/Users/nik00/py/proj/hyppi-independent.pkl")
df = df.sample(frac=1).reset_index(drop=True)[['Human','Yersinia','Joined','label']]   #SHUFFLE
df['label'] = np.where(df['label']==-1, 0, 1)
df

,Human,Yersinia,Joined,label
0,"[M, Y, V, T, M, M, M, T, D, Q, I, P, L, E, L, ...","[M, T, G, P, Q, Q, G, K, I, L, L, R, L, E, N, ...","[M, Y, V, T, M, M, M, T, D, Q, I, P, L, E, L, ...",1
1,"[M, E, D, H, Q, H, V, P, I, D, I, Q, T, S, K, ...","[M, K, N, R, T, L, G, S, I, F, I, V, A, G, T, ...","[M, E, D, H, Q, H, V, P, I, D, I, Q, T, S, K, ...",1
2,"[M, V, L, G, K, V, K, S, L, T, I, S, F, D, C, ...","[M, I, I, V, T, G, G, A, G, F, I, G, S, N, I, ...","[M, V, L, G, K, V, K, S, L, T, I, S, F, D, C, ...",1
3,"[M, E, H, K, E, V, V, L, L, L, L, L, F, L, K, ...","[M, L, I, K, L, L, T, K, V, F, G, S, R, N, D, ...","[M, E, H, K, E, V, V, L, L, L, L, L, F, L, K, ...",1
4,"[M, A, P, R, T, L, W, S, C, Y, L, C, C, L, L, ...","[M, T, N, T, S, Y, I, K, N, A, E, T, K, L, S, ...","[M, A, P, R, T, L, W, S, C, Y, L, C, C, L, L, ...",0
...,...,...,...,...
1509,"[M, G, T, P, A, S, G, R, K, R, T, P, V, K, D, ...","[M, I, K, L, G, I, V, M, D, P, I, S, S, I, N, ...","[M, G, T, P, A, S, G, R, K, R, T, P, V, K, D, ...",1
1510,"[M, R, S, A, A, A, A, P, R, S, P, A, V, A, T, ...","[M, R, S, C, L, S, G, T, R, C, V, P, R, R, L, ...","[M, R, S, A, A, A, A, P, R, S, P, A, V, A, T, ...",1
1511,"[M, L, L, R, L, L, L, A, W, A, A, A, G, P, T, ...","[M, R, V, E, I, G, N, M, S, G, D, D, R, E, P, ...","[M, L, L, R, L, L, L, A, W, A, A, A, G, P, T, ...",0
1512,"[M, T, L, E, E, L, V, A, C, D, N, A, A, Q, K, ...","[M, R, L, F, A, Q, L, G, W, Y, F, R, R, E, W, ...","[M, T, L, E, E, L, V, A, C, D, N, A, A, Q, K, ...",0


In [ ]:
df.to_pickle("C:/Users/nik00/py/proj/hyppi-independent.pkl")